In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
import numpy as np
import sys
sys.path
sys.executable
import pandas as pd 
import time
from csv import reader
#===================================For the Scatternet
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
readdatafolder = '/content/drive/MyDrive/GAN!/2tails_Sim.csv'
# output_folder = '/content/drive/MyDrive/inversepaper/resultsci06/new4w5percent/validate_lr/model_validate_lr'
numlayer = 4
n_hidden = 250
lr_rate = 0.005
lr_decay = 0.99
n_batch = 32
numEpochs = 500
import keras.losses
def my_loss_fn(y_true, y_pred):
    y_true_0 = y_true[0]
    y_pred_0 = y_pred[0]
    # print('y_true_0 is: ' + str(y_true_0))
    # print('y_pred_0 is: ' + str(y_pred_0))
    y_true_rest = y_true[1:]
    y_pred_rest = y_pred[1:]
    # mseloss = np.mean(np.abs(y_true_rest - y_pred_rest), axis=-1)
    maeloss = keras.metrics.mean_absolute_error(y_true_rest, y_pred_rest)
    zero_loss = keras.losses.mean_absolute_error(y_true_0,y_pred_0)
    return maeloss + zero_loss  # Note the `axis=-1`
keras.losses.my_loss_fn = my_loss_fn
def get_data(random_state=42):
    N1 = 10 #col11 omegaR
    N2 = 12 #col13 vel

    df = pd.read_csv(readdatafolder)#2tails_Sim_added.csv includes the data to be tested.

    #this takes the first row
    df.columns = ['col1','col2','col3','col4','col5','col6','col7','col8','col9','col10','col11','col12','col13','col14'] 
    x = df[['col1','col2','col3','col4','col5','col6','col7','col8','col9','col10','col11','col12','col13','col14']]
    xinput = df[['col1','col2','col3','col4','col5','col6','col7','col8']]
    xoutput = df[['col10','col11','col12','col13','col14']]
    # print(np.array(xoutput))
    xinput = np.array(xinput)


    # print(xoutput)
    x = np.array(x)
    x9 = df['col9']#omegahead
    x9 = abs(np.array(x9)) 
    x9 = np.reshape(x9, (x9.shape[0], 1))
    # print(x9)
    # print(x9.shape, np.array(xoutput).shape)
    xoutput = np.hstack((x9, np.array(xoutput)))
    # print(xoutput)


    #print(xinput.shape[0])#checked
    xnew = np.hstack((xinput, xoutput))
    # print(xnew.shape)
    # print(xnew)

    ##################Step 4 Multioutput regression with k-fold cross-validation
    ## get datasets from nondimensionalized variables
    L = xnew[:, 3] # tail length
    R = xnew[:, 4] # head radius
    r0 = xnew[:, 6] # tail radius
    x4 = L/R # element-wise division, checked
    x5 = L/r0
    eta_per = 4*np.pi*1/(np.log(2*x5) + 1/2)
    EI = 1e6*np.pi*r0**4/4
    x6 = xnew[:, 7]*eta_per*L**4/EI

    x4 = np.reshape(x4,(x4.shape[0],1))
    x5 = np.reshape(x5,(x5.shape[0],1))
    x6 = np.reshape(x6,(x6.shape[0],1))
    # xdata = np.hstack((addTailNum, xnew[:,:3], x4, x5, x6))
    xdata = xnew[:,:8]
    # print(xdata.shape)
    # print(xnew[:,8])
    ydata2 = xnew[:,12]
    # ydata1 = xnew[:,N1]*eta_per*L**4/EI #this is only for N1=10 omegaR
    # ydata2 = xnew[:,N2]*eta_per*L**3/EI #this is only for N2=12 vel
    # ydata1 = np.reshape(ydata1,(ydata1.shape[0],1))
    ydata2 = np.reshape(ydata2,(ydata2.shape[0],1))
    ydata = ydata2
    return xdata, ydata

def takeClosest(target,inputlist):
    closestvalue = min(inputlist,key=lambda x:abs(x-target))
    closestpos = inputlist.index(closestvalue)
    return closestvalue,closestpos
def main_design():
    global output_folder,numlayer
    train_X,train_Y = get_data()
    x_mean = [3.64753345762981,5.272309162307576,5.755964960394364,0.1613585719623448,0.007060981916006027,0.007060981916006027,0.005018542071439014,11.188341792706536]
    x_std = [2.866683912969323,2.2890653250896325,2.4740240246120173,0.06901035998872507,0.002955367277994693,0.002955367277994693,0.0006084509375476656,4.26861095566077]
    train_X -= x_mean
    train_X /= x_std
    # train_X, train_Y = train_X[:3], train_Y[:3]
    train_Y_flat = train_Y.flatten()
    wantedY = train_Y_flat[:70]
    # wantedY = np.array([0.00209625620378628])
    # wantedY = np.array([0.000974990513299457])
    # wantedY = np.array([0.125298148349653])

    x_size = 8   # Number of input dimension: 6
    y_size = train_Y.shape[1]   # Number of output dimension: 1
    y_number = train_Y.shape[0]
    writtenresult = []
    count = 0
    for eachY in wantedY:
        count = count + 1
        print("===========================Dealing with: " + str(count))
        # print("currently try to find parameter for velocity: " + str(eachY))
        _,closestpos = takeClosest(eachY,train_Y_flat.tolist())
        initialX = train_X[closestpos]
        # print("initialX is: " + str(initialX))

        randomX = np.random.rand(1,x_size)
        for param_index in range(x_size):
            initialrange = 0.2 * initialX[param_index]
            # print(initialX[param_index])
            # print(initialrange)
            initialpos = initialX[param_index] - 0.5 * initialrange
            randomX[:,param_index] = randomX[:,param_index] * initialrange + initialpos
        # print("Idealiz X: " + str(initialX))
        # print("initial X: " + str(randomX))
        initialize_X = randomX.flatten().reshape((1, x_size))
        # print(initialize_X.shape)

        sess = tf.compat.v1.InteractiveSession()
        x_placeholder =  tf.compat.v1.placeholder(tf.float32)
        x_var = tf.Variable(initialize_X, dtype=tf.float32)
        model = load_model('/content/drive/MyDrive/inversepaper/resultsci06/new4w5percent/validate_lr/model_validate_lr')
        # Check calling my_model
        assign_step = tf.compat.v1.assign(x_var, x_placeholder)
        sess.run(assign_step, feed_dict={x_placeholder: initialize_X})
        yhat = model(x_var) # This simple step is all I wanted!
        sess.run(yhat) # This outputs my_model's predicted value for input x_init_value
        # cost = tf.reduce_sum(tf.square(eachY-yhat))#+5.0*extra_cost*extra_cost
        costprecision = tf.reduce_sum(tf.square(eachY-yhat))
        costvariation = tf.reduce_sum(tf.square(x_var-initialX))
        cost = costprecision
        global_step = tf.Variable(0, trainable=False)
        # Now, define the objective function that has to be minimized

        optimizer = tf.compat.v1.train.AdamOptimizer().minimize(cost, var_list=[x_var])
                # Define the optimizer
        init = tf.compat.v1.global_variables_initializer()
        sess.run(init)
        # Run the optimization steps
        resultX = []
        resultY = -1
        
        for i in range(500): # for 50 steps
            sess.run(optimizer)
            loss = sess.run(cost)
            curr_yhat = sess.run(yhat)
            cost1 = sess.run(costprecision)
            cost2 = sess.run(costvariation)
            resultX = x_var.eval()[0]
            resultY = curr_yhat[0][0]
            # print("cost1 is: " + str(cost1) + " cost 2 is: " + str(cost2) + "ratio is: " + str(cost1/cost2))
            # print("step: " + str(i+1)+ ", loss: "+ str(loss))
            # print('Current X: '+str(resultX))  
            # print("Idealiz X: " + str(initialX))
            # print("initial X: " + str(randomX))
            # print("current y:" + str(resultY) + " idea y: " + str(eachY))
        print("result x: " + str(resultX))
        print("initialX: " + str(initialX))
        print("result y: " + str(resultY))
        print("ideal  y: " + str(eachY))
        resultX *= x_std
        resultX += x_mean
        appendinfo = resultX.tolist()
        appendinfo = appendinfo + [resultY, eachY]
        print(appendinfo)
        writtenresult.append(appendinfo)
        print(writtenresult)
    resultdataframe = pd.DataFrame(writtenresult)
    resultdataframe.to_csv('/content/drive/MyDrive/inversepaper/forward_lrrate.csv', index = False, sep=',', mode='a',columns=None)

if __name__ == "__main__":
    main_design()



===========================Dealing with: 1
result x: [ 0.4245465  -1.0854219   0.7520196  -0.37880093  1.3884463   1.2801259
 -1.1981453  -2.3049433 ]
initialX: [ 0.39737082 -1.09310081  0.74745638 -0.30862862  1.40727622  1.40727622
 -1.11519603 -2.41068861]
result y: 9.8780794e-05
ideal  y: 1.168546863e-05
[4.864573955535889, 2.7877073287963867, 7.616479396820068, 0.13521738350391388, 0.011164351366460323, 0.010844224132597446, 0.004289529286324978, 1.3494354486465454, 9.8780794e-05, 1.168546863e-05]
[[4.864573955535889, 2.7877073287963867, 7.616479396820068, 0.13521738350391388, 0.011164351366460323, 0.010844224132597446, 0.004289529286324978, 1.3494354486465454, 9.8780794e-05, 1.168546863e-05]]
===========================Dealing with: 2


/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1771: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


result x: [ 0.4511994  -1.0531979   0.68982244 -0.33004856  1.2679329   1.2757097
 -1.0142767  -2.2228203 ]
initialX: [ 0.39737082 -1.09310081  0.74745638 -0.30862862  1.40727622  1.40727622
 -1.11519603 -2.34873637]
result y: 9.850932e-05
ideal  y: 1.946624402e-05
[4.940979480743408, 2.8614704608917236, 7.462602138519287, 0.1385817974805832, 0.010808189399540424, 0.010831172578036785, 0.004401404410600662, 1.6999870538711548, 9.850932e-05, 1.946624402e-05]
[[4.864573955535889, 2.7877073287963867, 7.616479396820068, 0.13521738350391388, 0.011164351366460323, 0.010844224132597446, 0.004289529286324978, 1.3494354486465454, 9.8780794e-05, 1.168546863e-05], [4.940979480743408, 2.8614704608917236, 7.462602138519287, 0.1385817974805832, 0.010808189399540424, 0.010831172578036785, 0.004401404410600662, 1.6999870538711548, 9.850932e-05, 1.946624402e-05]]
===========================Dealing with: 3
result x: [ 0.4421142 -1.1782414  0.7591178 -0.3270066  1.3695806  1.4823923
 -0.9925261 -2.142399